## Phase 2: Exploratory Data Analysis (EDA)

This notebook delivers the Phase 2 analyses:
- Task 2.1: Utilization Pattern Analysis (dashboard of 5–7 visualizations)
- Task 2.2: Demographic Insights Discovery
- Task 2.3: Cost Efficiency and Subcategory Analysis

Data sources used:
- `data/usage_data.csv`, `data/employee_data.csv`, `data/benefits_data.csv`, `data/feedback_data.csv`
- Consolidated: `data/cleaned_data.csv`

Note: Visualizations use Plotly with distinct designs versus the reference notebook to provide fresh perspectives while maintaining clarity.


In [1]:
# Imports and setup
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

pd.options.display.float_format = lambda x: f"{x:,.2f}"

# Paths
USAGE_PATH = "data/usage_data.csv"
EMP_PATH = "data/employee_data.csv"
BEN_PATH = "data/benefits_data.csv"
FEED_PATH = "data/feedback_data.csv"
CLEAN_PATH = "data/cleaned_data.csv"

# Load data
usage = pd.read_csv(USAGE_PATH, parse_dates=["LastUsedDate"], infer_datetime_format=True)
employees = pd.read_csv(EMP_PATH)
benefits = pd.read_csv(BEN_PATH)
feedback = pd.read_csv(FEED_PATH)
cleaned = pd.read_csv(CLEAN_PATH, parse_dates=["LastUsedDate"], infer_datetime_format=True)

# Basic checks
for name, df in {
    "usage": usage, "employees": employees, "benefits": benefits, "feedback": feedback, "cleaned": cleaned
}.items():
    print(f"{name}: {df.shape} | cols: {list(df.columns)[:8]}...")

# Join base (left join usage with dims)
base = (usage
    .merge(benefits, on="BenefitID", how="left")
    .merge(employees, on="EmployeeID", how="left")
    .merge(feedback, on=["EmployeeID","BenefitID"], how="left")
)

# Ensure types
base["UsageFrequency"] = pd.to_numeric(base["UsageFrequency"], errors="coerce")
base["BenefitCost"] = pd.to_numeric(base["BenefitCost"], errors="coerce")
base["SatisfactionScore"] = pd.to_numeric(base.get("SatisfactionScore", np.nan), errors="coerce")

# Derive time fields
base["year"] = base["LastUsedDate"].dt.year
base["month"] = base["LastUsedDate"].dt.month
base["year_month"] = base["LastUsedDate"].dt.to_period("M").astype(str)

# Engagement buckets - corrected to match requirements exactly
# Low: 0-2, Medium: 3-6, High: ≥7
bins = [-np.inf, 2.5, 6.5, np.inf]  # Using .5 to ensure correct categorization
labels = ["Low (0–2)", "Medium (3–6)", "High (≥7)"]
base["Engagement"] = pd.cut(base["UsageFrequency"], bins=bins, labels=labels)

base.head(3)


/Users/jackyhe/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


usage: (50000, 4) | cols: ['EmployeeID', 'BenefitID', 'UsageFrequency', 'LastUsedDate']...
employees: (5000, 5) | cols: ['EmployeeID', 'Age', 'Gender', 'Department', 'Tenure']...
benefits: (30, 4) | cols: ['BenefitID', 'BenefitType', 'BenefitSubType', 'BenefitCost']...
feedback: (30000, 4) | cols: ['EmployeeID', 'BenefitID', 'SatisfactionScore', 'Comments']...
cleaned: (7626, 45) | cols: ['EmployeeID', 'BenefitID', 'UsageFrequency', 'LastUsedDate', 'Age', 'Gender', 'Department', 'Tenure']...


/var/folders/kk/c79wsldj45df2gtvh0ydth9m0000gn/T/ipykernel_14210/2330095537.py:18: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  usage = pd.read_csv(USAGE_PATH, parse_dates=["LastUsedDate"], infer_datetime_format=True)
/var/folders/kk/c79wsldj45df2gtvh0ydth9m0000gn/T/ipykernel_14210/2330095537.py:22: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  cleaned = pd.read_csv(CLEAN_PATH, parse_dates=["LastUsedDate"], infer_datetime_format=True)


,EmployeeID,BenefitID,UsageFrequency,LastUsedDate,BenefitType,BenefitSubType,BenefitCost,Age,Gender,Department,Tenure,SatisfactionScore,Comments,year,month,year_month,Engagement
0,4245,25,2,2023-09-01,Life Insurance,Supplemental Standard,774.91,60,Female,Finance,5,NaN,NaN,2023,9,2023-09,Low (0–2)
1,220,20,4,2024-05-03,Tuition Reimbursement,Undergraduate Degree,489.96,64,Male,HR,35,1.00,Not worth the hassle.,2024,5,2024-05,Medium (3–6)
2,4716,24,4,2024-02-10,Gym Membership,Tier 3 Partners,780.78,46,Female,IT,8,NaN,NaN,2024,2,2024-02,Medium (3–6)


### Task 2.1: Utilization Pattern Analysis
We explore overall utilization, temporal trends, engagement bands, and utilization by subcategory. All visuals use Plotly with clear, minimal styling and consistent color palettes.


In [2]:
# 2.1.1 Most/least utilized benefits by UsageFrequency
util_by_subtype = (base
    .groupby(["BenefitType", "BenefitSubType"], dropna=False)["UsageFrequency"]
    .sum()
    .reset_index()
    .sort_values("UsageFrequency", ascending=False)
)

# Top 10 & Bottom 10
top10 = util_by_subtype.head(10)
bottom10 = util_by_subtype.tail(10)

fig_top = px.bar(
    top10,
    x="UsageFrequency", y="BenefitSubType",
    color="BenefitType",
    orientation="h",
    title="Top 10 Most Utilized Benefit Subtypes",
)
fig_top.update_layout(yaxis=dict(categoryorder="total ascending"))

fig_bottom = px.bar(
    bottom10,
    x="UsageFrequency", y="BenefitSubType",
    color="BenefitType",
    orientation="h",
    title="Bottom 10 Least Utilized Benefit Subtypes",
)
fig_bottom.update_layout(yaxis=dict(categoryorder="total ascending"))

fig_top.show()
fig_bottom.show()

util_by_subtype.head(10)


/Users/jackyhe/anaconda3/lib/python3.11/site-packages/plotly/express/_core.py:1979: FutureWarning: When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.
  sf: grouped.get_group(s if len(s) > 1 else s[0])
/Users/jackyhe/anaconda3/lib/python3.11/site-packages/plotly/express/_core.py:1979: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



,BenefitType,BenefitSubType,UsageFrequency
20,Retirement Plan,401k High Contribution,6260
28,Tuition Reimbursement,Undergraduate Degree,6115
16,Life Insurance,Supplemental Standard,6075
5,Gym Membership,Family Membership,6069
10,Health Insurance,HMO Family,6017
26,Tuition Reimbursement,Individual Courses,5918
17,Professional Development,Conference Attendance,5828
23,Retirement Plan,401k Standard Matching,5813
24,Technology Stipend,Monthly Internet Allowance,5803
18,Retirement Plan,401k Basic Matching,5802


In [3]:
# 2.1.2 Temporal trends (monthly for 2023–2024)
monthly = (base
    .query("year in [2023, 2024]")
    .groupby(["year", "year_month"], dropna=False)["UsageFrequency"]
    .sum()
    .reset_index()
)

fig_monthly = px.line(
    monthly,
    x="year_month", y="UsageFrequency",
    color="year", markers=True,
    title="Monthly Usage Frequency (2023–2024)",
)
fig_monthly.update_layout(xaxis_title="Year-Month", legend_title_text="Year")
fig_monthly.show()

monthly.head(6)


/Users/jackyhe/anaconda3/lib/python3.11/site-packages/plotly/express/_core.py:1979: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



,year,year_month,UsageFrequency
0,2023,2023-08,12364
1,2023,2023-09,13593
2,2023,2023-10,14307
3,2023,2023-11,13961
4,2023,2023-12,14366
5,2024,2024-01,14453


In [4]:
# 2.1.3 Engagement categorization and distribution
engagement_counts = base["Engagement"].value_counts(dropna=False).rename_axis("Engagement").reset_index(name="Count")
fig_eng_dist = px.pie(
    engagement_counts, values="Count", names="Engagement",
    title="Engagement Distribution (by UsageFrequency)", hole=0.45
)
fig_eng_dist.show()

# Engagement by BenefitSubType
engagement_by_subtype = (base
    .groupby(["BenefitSubType", "Engagement"], dropna=False)["EmployeeID"]
    .nunique()
    .reset_index(name="UniqueEmployees")
)
fig_eng_sub = px.bar(
    engagement_by_subtype,
    x="BenefitSubType", y="UniqueEmployees", color="Engagement",
    title="Unique Employees by Engagement Level and Benefit Subtype"
)
fig_eng_sub.update_layout(xaxis_tickangle=-35)
fig_eng_sub.show()

engagement_counts


/var/folders/kk/c79wsldj45df2gtvh0ydth9m0000gn/T/ipykernel_14210/2231621303.py:11: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

/Users/jackyhe/anaconda3/lib/python3.11/site-packages/plotly/express/_core.py:1958: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

/Users/jackyhe/anaconda3/lib/python3.11/site-packages/plotly/express/_core.py:1979: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



,Engagement,Count
0,Low (0–2),24188
1,Medium (3–6),16945
2,High (≥7),9803


In [5]:
# 2.1.4 Utilization by BenefitSubType (focus on 401k variants)
sub_focus = base[base["BenefitSubType"].str.contains("401k", na=False)]
sub_util = (sub_focus
    .groupby(["BenefitSubType"], dropna=False)["UsageFrequency"]
    .sum()
    .reset_index()
    .sort_values("UsageFrequency", ascending=False)
)
fig_401k = px.bar(
    sub_util, x="BenefitSubType", y="UsageFrequency",
    title="Utilization by 401k Subtypes"
)
fig_401k.update_layout(xaxis_tickangle=-30)
fig_401k.show()

sub_util.head()


,BenefitSubType,UsageFrequency
2,401k High Contribution,6260
5,401k Standard Matching,5813
0,401k Basic Matching,5802
1,401k Catch-Up Contributions,5583
3,401k Investment Fees,5559


In [6]:
# 2.1.5 Dashboard summary (treemap + scatter)
# Treemap: Usage by BenefitType/SubType
fig_tree = px.treemap(
    util_by_subtype, path=["BenefitType", "BenefitSubType"], values="UsageFrequency",
    title="Treemap: Usage Frequency by Benefit Type and Subtype"
)
fig_tree.show()

# Scatter: BenefitCost vs UsageFrequency (summed by subtype)
cost_by_subtype = (base
    .groupby(["BenefitType", "BenefitSubType"], dropna=False)
    .agg(UsageFrequency=("UsageFrequency", "sum"), AvgCost=("BenefitCost", "mean"))
    .reset_index()
)
fig_scatter = px.scatter(
    cost_by_subtype,
    x="AvgCost", y="UsageFrequency", color="BenefitType", hover_name="BenefitSubType",
    title="Cost vs. Usage (Subtype-Level)",
    size="UsageFrequency"
)
fig_scatter.show()

cost_by_subtype.head()


/Users/jackyhe/anaconda3/lib/python3.11/site-packages/plotly/express/_core.py:1979: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



,BenefitType,BenefitSubType,UsageFrequency,AvgCost
0,Cell Phone Allowance,Monthly Communications,5374,65.00
1,Childcare,After-School Care,5745,475.00
2,Childcare,On-Site Infant Care,5359,915.48
3,Commuter Benefits,Transit Subsidy,5627,325.00
4,Flexible Spending Account,Healthcare FSA,5759,450.00


### Task 2.2: Demographic Insights Discovery
We analyze benefit usage across Age, Gender, Department, and Tenure, along with preferences by subcategory.


In [7]:
# 2.2.1 Usage by demographics
# Age groups for clarity
age_bins = [0, 25, 35, 45, 55, 200]
age_labels = ["<=25", "26–35", "36–45", "46–55", "56+"]
base["AgeGroup"] = pd.cut(base["Age"], bins=age_bins, labels=age_labels, right=True)

# Tenure groups (years)
tenure_bins = [0, 2, 5, 10, 20, 100]
tenure_labels = ["<2", "2–5", "5–10", "10–20", "20+"]
base["TenureGroup"] = pd.cut(base["Tenure"], bins=tenure_bins, labels=tenure_labels, right=True)

# Usage by Gender and Department
usage_gender_dept = (base
    .groupby(["Gender", "Department"], dropna=False)["UsageFrequency"]
    .sum()
    .reset_index()
)
fig_heat_gd = px.density_heatmap(
    usage_gender_dept, x="Department", y="Gender", z="UsageFrequency",
    color_continuous_scale="Viridis",
    title="Usage Frequency by Gender and Department"
)
fig_heat_gd.show()

# Usage by AgeGroup and Department
usage_age_dept = (base
    .groupby(["AgeGroup", "Department"], dropna=False)["UsageFrequency"]
    .sum().reset_index()
)
fig_heat_ad = px.density_heatmap(
    usage_age_dept, x="Department", y="AgeGroup", z="UsageFrequency",
    color_continuous_scale="Magma",
    title="Usage Frequency by Age Group and Department"
)
fig_heat_ad.show()

usage_gender_dept.head()


/var/folders/kk/c79wsldj45df2gtvh0ydth9m0000gn/T/ipykernel_14210/2858639992.py:27: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



,Gender,Department,UsageFrequency
0,Female,Finance,17604
1,Female,HR,11656
2,Female,IT,19835
3,Female,Marketing,16491
4,Female,Sales,16636


In [8]:
# 2.2.2 Preferences by BenefitSubType (e.g., IT’s preference for Technology Stipend)
# Compute average UsageFrequency per Department/Subtype
pref_dept_subtype = (base
    .groupby(["Department", "BenefitSubType"], dropna=False)
    .agg(avg_usage=("UsageFrequency", "mean"), users=("EmployeeID", "nunique"))
    .reset_index()
)

# Highlight departments' top 5 subtypes by avg usage
pref_top = pref_dept_subtype.sort_values(["Department", "avg_usage"], ascending=[True, False]).groupby("Department").head(5)
fig_pref = px.bar(
    pref_top,
    x="BenefitSubType", y="avg_usage", color="Department",
    title="Top 5 Subtype Preferences per Department (by Avg Usage)", barmode="group"
)
fig_pref.update_layout(xaxis_tickangle=-35)
fig_pref.show()

pref_top.head(10)


/Users/jackyhe/anaconda3/lib/python3.11/site-packages/plotly/express/_core.py:1979: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



,Department,BenefitSubType,avg_usage,users
24,Finance,Supplemental Standard,3.71,291
23,Finance,Supplemental High Amount,3.71,295
2,Finance,401k High Contribution,3.59,302
29,Finance,Undergraduate Degree,3.54,315
6,Finance,After-School Care,3.52,290
42,HR,HDHP Individual,3.73,211
40,HR,Family Membership,3.70,211
36,HR,After-School Care,3.61,191
57,HR,Tier 3 Partners,3.60,212
45,HR,Individual Courses,3.60,214


In [9]:
# 2.2.3 Stacked bar: Usage by BenefitType split by Gender and AgeGroup
usage_type_gender = (base
    .groupby(["BenefitType", "Gender"], dropna=False)["UsageFrequency"].sum().reset_index()
)
fig_stacked_gender = px.bar(
    usage_type_gender, x="BenefitType", y="UsageFrequency", color="Gender",
    title="Usage by Benefit Type split by Gender", barmode="stack"
)
fig_stacked_gender.update_layout(xaxis_tickangle=-25)
fig_stacked_gender.show()

usage_type_age = (base
    .groupby(["BenefitType", "AgeGroup"], dropna=False)["UsageFrequency"].sum().reset_index()
)
fig_stacked_age = px.bar(
    usage_type_age, x="BenefitType", y="UsageFrequency", color="AgeGroup",
    title="Usage by Benefit Type split by Age Group", barmode="stack"
)
fig_stacked_age.update_layout(xaxis_tickangle=-25)
fig_stacked_age.show()


/Users/jackyhe/anaconda3/lib/python3.11/site-packages/plotly/express/_core.py:1979: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



/var/folders/kk/c79wsldj45df2gtvh0ydth9m0000gn/T/ipykernel_14210/1164380290.py:13: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

/Users/jackyhe/anaconda3/lib/python3.11/site-packages/plotly/express/_core.py:1958: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

/Users/jackyhe/anaconda3/lib/python3.11/site-packages/plotly/express/_core.py:1979: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



### Task 2.3: Cost Efficiency and Subcategory Analysis
We compute cost-per-usage, an ROI score combining cost and satisfaction, and identify high-cost underutilized subcategories. Visuals include quadrant plots for cost vs usage and cost vs satisfaction.


In [10]:
# 2.3.1 Cost-per-usage by BenefitID / BenefitSubType
base["usage_nonzero"] = base["UsageFrequency"].replace({0: np.nan})
base["cost_per_usage"] = base["BenefitCost"] / base["usage_nonzero"]

cpu_by_subtype = (base
    .groupby(["BenefitType", "BenefitSubType"], dropna=False)
    .agg(
        avg_cost=("BenefitCost", "mean"),
        total_usage=("UsageFrequency", "sum"),
        cpu_mean=("cost_per_usage", "mean"),
        satisfaction_mean=("SatisfactionScore", "mean"),
        members=("EmployeeID", "nunique")
    )
    .reset_index()
)

cpu_by_subtype.head(10)


,BenefitType,BenefitSubType,avg_cost,total_usage,cpu_mean,satisfaction_mean,members
0,Cell Phone Allowance,Monthly Communications,65.00,5374,23.24,3.10,1400
1,Childcare,After-School Care,475.00,5745,163.20,3.12,1426
2,Childcare,On-Site Infant Care,915.48,5359,322.28,2.91,1371
3,Commuter Benefits,Transit Subsidy,325.00,5627,118.96,3.08,1446
4,Flexible Spending Account,Healthcare FSA,450.00,5759,153.19,3.24,1442
5,Gym Membership,Family Membership,519.66,6069,178.38,3.05,1464
6,Gym Membership,Tier 1 Partners,73.36,5275,26.20,3.12,1372
7,Gym Membership,Tier 2 Partners,343.73,5434,120.93,3.07,1417
8,Gym Membership,Tier 3 Partners,780.78,5688,264.48,2.92,1439
9,Health Insurance,HDHP Individual,84.55,5788,29.63,3.13,1429


In [11]:
# 2.3.2 ROI score (normalize cost-per-usage and SatisfactionScore)
# Normalize to 0-1 range
for col in ["cpu_mean", "satisfaction_mean"]:
    col_min = cpu_by_subtype[col].min()
    col_max = cpu_by_subtype[col].max()
    cpu_by_subtype[f"{col}_norm"] = (cpu_by_subtype[col] - col_min) / (col_max - col_min) if col_max > col_min else 0

# Lower cost-per-usage is better, higher satisfaction is better
cpu_by_subtype["roi_score"] = (1 - cpu_by_subtype["cpu_mean_norm"]) * 0.6 + cpu_by_subtype["satisfaction_mean_norm"] * 0.4

cpu_by_subtype.sort_values("roi_score", ascending=False).head(10)


,BenefitType,BenefitSubType,avg_cost,total_usage,cpu_mean,satisfaction_mean,members,cpu_mean_norm,satisfaction_mean_norm,roi_score
9,Health Insurance,HDHP Individual,84.55,5788,29.63,3.13,1429,0.02,0.76,0.89
6,Gym Membership,Tier 1 Partners,73.36,5275,26.20,3.12,1372,0.01,0.73,0.89
0,Cell Phone Allowance,Monthly Communications,65.00,5374,23.24,3.10,1400,0.00,0.68,0.87
29,Wellness Programs,Premium Discount Tier 1,125.00,5544,44.45,3.08,1394,0.07,0.64,0.82
24,Technology Stipend,Monthly Internet Allowance,75.00,5803,26.64,3.01,1447,0.01,0.47,0.78
4,Flexible Spending Account,Healthcare FSA,450.00,5759,153.19,3.24,1442,0.41,1.00,0.75
14,Life Insurance,Dependent Coverage,165.54,5652,58.73,3.03,1388,0.11,0.53,0.75
27,Tuition Reimbursement,Professional Certification,359.21,5453,121.89,3.11,1373,0.31,0.69,0.69
3,Commuter Benefits,Transit Subsidy,325.00,5627,118.96,3.08,1446,0.30,0.64,0.68
7,Gym Membership,Tier 2 Partners,343.73,5434,120.93,3.07,1417,0.31,0.63,0.67


In [12]:
# 2.3.3 Identify underutilized high-cost subcategories
# Define thresholds using medians to avoid outlier bias
cost_thresh = cpu_by_subtype["avg_cost"].median()
usage_thresh = cpu_by_subtype["total_usage"].median()

cpu_by_subtype["underutilized_high_cost"] = (cpu_by_subtype["avg_cost"] > cost_thresh) & (cpu_by_subtype["total_usage"] < usage_thresh)

candidates = cpu_by_subtype[cpu_by_subtype["underutilized_high_cost"]].sort_values("avg_cost", ascending=False)
candidates.head(15)


,BenefitType,BenefitSubType,avg_cost,total_usage,cpu_mean,satisfaction_mean,members,cpu_mean_norm,satisfaction_mean_norm,roi_score,underutilized_high_cost
15,Life Insurance,Supplemental High Amount,969.28,5515,340.47,2.98,1380,1.00,0.42,0.17,True
11,Health Insurance,PPO Family,928.75,5461,323.96,3.07,1398,0.95,0.61,0.27,True
2,Childcare,On-Site Infant Care,915.48,5359,322.28,2.91,1371,0.94,0.25,0.13,True
22,Retirement Plan,401k Maximum Matching,913.01,5396,319.23,2.95,1362,0.93,0.34,0.18,True
25,Tuition Reimbursement,Graduate Degree,824.53,5495,282.03,3.06,1399,0.82,0.60,0.35,True
21,Retirement Plan,401k Investment Fees,743.01,5559,247.87,3.09,1373,0.71,0.66,0.44,True
12,Health Insurance,PPO Individual,706.93,5347,242.81,3.15,1393,0.69,0.80,0.51,True


In [13]:
# 2.3.4 Quadrant plots
# A) Cost vs Usage with quadrant medians
fig_quad1 = px.scatter(
    cpu_by_subtype, x="avg_cost", y="total_usage", color="underutilized_high_cost",
    hover_data=["BenefitType", "BenefitSubType"], title="Quadrant: Cost vs Usage"
)
fig_quad1.add_hline(y=usage_thresh, line_dash="dash", line_color="gray")
fig_quad1.add_vline(x=cost_thresh, line_dash="dash", line_color="gray")
fig_quad1.show()

# B) Cost-per-usage vs Satisfaction
cpu_by_subtype_nonan = cpu_by_subtype.dropna(subset=["cpu_mean", "satisfaction_mean"]).copy()
cpu_thresh = cpu_by_subtype_nonan["cpu_mean"].median()
sat_thresh = cpu_by_subtype_nonan["satisfaction_mean"].median()

fig_quad2 = px.scatter(
    cpu_by_subtype_nonan, x="cpu_mean", y="satisfaction_mean", color="BenefitType",
    hover_name="BenefitSubType", title="Quadrant: Cost-per-Usage vs Satisfaction"
)
fig_quad2.add_hline(y=sat_thresh, line_dash="dash", line_color="gray")
fig_quad2.add_vline(x=cpu_thresh, line_dash="dash", line_color="gray")
fig_quad2.show()


/Users/jackyhe/anaconda3/lib/python3.11/site-packages/plotly/express/_core.py:1979: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



/Users/jackyhe/anaconda3/lib/python3.11/site-packages/plotly/express/_core.py:1979: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



In [14]:
# 2.3.5 Rank benefits by cost efficiency and flag elimination candidates
ranked = cpu_by_subtype.sort_values(["roi_score", "total_usage"], ascending=[False, False])
ranked["eliminate_candidate"] = ranked["underutilized_high_cost"] & (ranked["roi_score"] < ranked["roi_score"].median())

fig_rank = px.bar(
    ranked.head(25), x="BenefitSubType", y="roi_score", color="eliminate_candidate",
    title="Top 25 Subtypes by ROI Score (flagging elimination candidates)"
)
fig_rank.update_layout(xaxis_tickangle=-35)
fig_rank.show()

ranked.head(15)


/Users/jackyhe/anaconda3/lib/python3.11/site-packages/plotly/express/_core.py:1979: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



,BenefitType,BenefitSubType,avg_cost,total_usage,cpu_mean,satisfaction_mean,members,cpu_mean_norm,satisfaction_mean_norm,roi_score,underutilized_high_cost,eliminate_candidate
9,Health Insurance,HDHP Individual,84.55,5788,29.63,3.13,1429,0.02,0.76,0.89,False,False
6,Gym Membership,Tier 1 Partners,73.36,5275,26.20,3.12,1372,0.01,0.73,0.89,False,False
0,Cell Phone Allowance,Monthly Communications,65.00,5374,23.24,3.10,1400,0.00,0.68,0.87,False,False
29,Wellness Programs,Premium Discount Tier 1,125.00,5544,44.45,3.08,1394,0.07,0.64,0.82,False,False
24,Technology Stipend,Monthly Internet Allowance,75.00,5803,26.64,3.01,1447,0.01,0.47,0.78,False,False
4,Flexible Spending Account,Healthcare FSA,450.00,5759,153.19,3.24,1442,0.41,1.00,0.75,False,False
14,Life Insurance,Dependent Coverage,165.54,5652,58.73,3.03,1388,0.11,0.53,0.75,False,False
27,Tuition Reimbursement,Professional Certification,359.21,5453,121.89,3.11,1373,0.31,0.69,0.69,False,False
3,Commuter Benefits,Transit Subsidy,325.00,5627,118.96,3.08,1446,0.30,0.64,0.68,False,False
7,Gym Membership,Tier 2 Partners,343.73,5434,120.93,3.07,1417,0.31,0.63,0.67,False,False


### Notes and Interpretation
- Utilization dashboard highlights top/least used subcategories and temporal peaks.
- Engagement bands reveal distribution of frequent vs infrequent users and which subtypes skew high-engagement.
- Demographics heatmaps emphasize department- and age-driven usage patterns.
- ROI combines efficiency (low cost-per-usage) and satisfaction, surfacing best bets.
- Quadrants and rankings flag underutilized high-cost options that may be candidates for streamlining.

You can export figures to static images if needed using Plotly's `write_image` with `kaleido`.
